In [165]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mlos_core.optimizers
import ConfigSpace as CS
from ConfigSpace import UniformIntegerHyperparameter
from ConfigSpace import UniformFloatHyperparameter
from ConfigSpace import CategoricalHyperparameter

In [166]:
def cost_model(index):
    performance_data = pd.read_csv('../../build/performance/performance_metrics.csv', header=None, names=["empty_reads", "non_empty_reads", "range_reads", "writes"])
    return performance_data.iloc[index:].mean().sum()

In [167]:
input_space = CS.ConfigurationSpace(seed=1234)

max_open_files=CS.Integer("max_open_files",(-1,1000),default=-1)
max_total_wal_size=CS.Integer("max_total_wal_size",(0,1000))
delete_obsolete_files_period_micros=CS.Integer("delete_obsolete_files_period_micros",(1,1000 * 60 * 60 * 1000000),default=6 * 60 * 60 * 1000000)

max_background_jobs=CS.Integer("max_background_jobs", (1,20), default=2)
max_subcompactions=CS.Integer("max_subcompactions", (1,20), default=1)
compaction_readahead_size=CS.Integer("compaction_readahead_size",(0,1000 * 1024 * 1024),default=2 * 1024 * 1024)

writable_file_max_buffer_size=CS.Integer("writable_file_max_buffer_size",(0,1000*1024*1024),default=1024 * 1024)
delayed_write_rate=CS.Integer("delayed_write_rate",(0,1000*1024*1024),default=0)
avoid_flush_during_shutdown=CS.Categorical("avoid_flush_during_shutdown", ["true","false"], default="true") 

max_write_buffer_number=CS.Integer("max_write_buffer_number", (1,20), default=2)
inplace_update_num_locks=CS.Integer("inplace_update_num_locks",(0,100000),default=10000)
memtable_prefix_bloom_size_ratio=CS.Float("memtable_prefix_bloom_size_ratio",(0.0,1.0),default=0)

memtable_whole_key_filtering=CS.Categorical("memtable_whole_key_filtering",["true","false"], default="false") 
memtable_huge_page_size=CS.Integer("memtable_huge_page_size",(0,1000 * 1024 * 1024),default=0)
arena_block_size=CS.Integer("arena_block_size",(0, 2<<30),default=0)

# compression_per_level=CS.("compression_per_level")
level0_slowdown_writes_trigger=CS.Integer("level0_slowdown_writes_trigger", (1,25), default=20) 
level0_stop_writes_trigger=CS.Integer("level0_stop_writes_trigger", (1,50), default=36) 

target_file_size_base=CS.Integer("target_file_size_base", (1,500 * 1048576), default=64 * 1048576)
target_file_size_multiplier=CS.Integer("target_file_size_multiplier", (1,20), default=1)
max_bytes_for_level_multiplier=CS.Float("max_bytes_for_level_multiplier", (2,20.0), default=10)

# max_bytes_for_level_multiplier_additional=CS.("max_bytes_for_level_multiplier_additional")
# max_compaction_bytes=CS.Integer("max_compaction_bytes")
# soft_pending_compaction_bytes_limit=CS.Integer("soft_pending_compaction_bytes_limit")

# hard_pending_compaction_bytes_limit=CS.Integer("hard_pending_compaction_bytes_limit")
# compaction_options_universal=CS.("compaction_options_universal")
# compaction_options_fifo=CS.("compaction_options_fifo")

max_sequential_skip_in_iterations=CS.Integer("max_sequential_skip_in_iterations",(0,999999),default=8)
max_successive_merges=CS.Integer("max_successive_merges",(0,1000),default=0)
check_flush_compaction_key_order=CS.Categorical("check_flush_compaction_key_order",["true","false"], default="true") 

paranoid_file_checks=CS.Categorical("paranoid_file_checks",["true","false"], default="false") 
report_bg_io_stats=CS.Categorical("report_bg_io_stats",["true","false"], default="false") 
# ttl=CS.Integer("ttl")

# periodic_compaction_seconds=CS.Integer("periodic_compaction_seconds")
# bottommost_temperature=CS.("bottommost_temperature")
enable_blob_files=CS.Categorical("enable_blob_files",["true","false"], default="false") 

min_blob_size=CS.Integer("min_blob_size",(0,1024),default=0)
# blob_file_size=CS.Integer("blob_file_size")
# blob_compression_type=CS.("blob_compression_type")

enable_blob_garbage_collection=CS.Categorical("enable_blob_garbage_collection",["true","false"], default="false") 
blob_garbage_collection_age_cutoff=CS.Float("blob_garbage_collection_age_cutoff",(0.0,1.0),default=0.25)
blob_garbage_collection_force_threshold=CS.Float("blob_garbage_collection_force_threshold",(0.0,1.0),default=1.00)

blob_compaction_readahead_size=CS.Integer("blob_compaction_readahead_size",(0,999999),default=0)
blob_file_starting_level=CS.Integer("blob_file_starting_level",(0,20),default=0)

In [168]:
input_space.add_hyperparameters([max_background_jobs,
                                max_open_files,
                                 max_total_wal_size,
                                 delete_obsolete_files_period_micros,
                                max_subcompactions,
                                compaction_readahead_size,
                                writable_file_max_buffer_size,
                                delayed_write_rate,
                                avoid_flush_during_shutdown,
                                max_write_buffer_number,
                                inplace_update_num_locks,
                                memtable_prefix_bloom_size_ratio,
                                memtable_whole_key_filtering,
                                memtable_huge_page_size,
                                arena_block_size,
                                level0_slowdown_writes_trigger,
                                level0_stop_writes_trigger,
                                target_file_size_base,
                                target_file_size_multiplier,
                                max_bytes_for_level_multiplier,
                                max_sequential_skip_in_iterations,
                                max_successive_merges,
                                check_flush_compaction_key_order,
                                paranoid_file_checks,
                                report_bg_io_stats,
                                enable_blob_files,
                                min_blob_size,
                                enable_blob_garbage_collection,
                                blob_garbage_collection_age_cutoff,
                                blob_garbage_collection_force_threshold,
                                blob_compaction_readahead_size,
                                blob_file_starting_level])

[max_background_jobs, Type: UniformInteger, Range: [1, 20], Default: 2,
 max_open_files, Type: UniformInteger, Range: [-1, 1000], Default: -1,
 max_total_wal_size, Type: UniformInteger, Range: [0, 1000], Default: 500,
 delete_obsolete_files_period_micros, Type: UniformInteger, Range: [1, 3600000000000], Default: 21600000000,
 max_subcompactions, Type: UniformInteger, Range: [1, 20], Default: 1,
 compaction_readahead_size, Type: UniformInteger, Range: [0, 1048576000], Default: 2097152,
 writable_file_max_buffer_size, Type: UniformInteger, Range: [0, 1048576000], Default: 1048576,
 delayed_write_rate, Type: UniformInteger, Range: [0, 1048576000], Default: 0,
 avoid_flush_during_shutdown, Type: Categorical, Choices: {true, false}, Default: true,
 max_write_buffer_number, Type: UniformInteger, Range: [1, 20], Default: 2,
 inplace_update_num_locks, Type: UniformInteger, Range: [0, 100000], Default: 10000,
 memtable_prefix_bloom_size_ratio, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.0

In [169]:
#optimizer = mlos_core.optimizers.RandomOptimizer(parameter_space=input_space)

#optimizer = mlos_core.optimizers.FlamlOptimizer(parameter_space=input_space)

optimizer = mlos_core.optimizers.SmacOptimizer(parameter_space=input_space) # , seed=42, n_random_init=20)

In [170]:
optimizer

SmacOptimizer(space_adapter=None)

In [171]:
import os
import time
import sys
def pass_values_to_interface(new_val):
    pipe_path = '../../build/passing_params_pipe'
    if not os.path.exists(pipe_path):
        os.mkfifo(pipe_path)
    pipe_fd = os.open(pipe_path, os.O_RDWR | os.O_TRUNC)
    data =''
    try:
        for x in new_val:
            data+='\n' + x + '=' + str(new_val[x].iloc[0])
        data=data.encode('utf-8')
        os.write(pipe_fd, data)
        sys.stdout.flush()
    except KeyboardInterrupt:
        pass
#     finally:
#         os.close(pipe_fd)


In [172]:
def read_epochs_pipe():
    while True:
        epochs_pipe_path = '../../build/passing_epochs'
        if os.path.exists(epochs_pipe_path):
            epochs_pipe_fd = os.open(epochs_pipe_path, os.O_RDONLY | os.O_TRUNC)
            try:
                buffer_size = 10000
                buffer = os.read(epochs_pipe_fd, buffer_size).decode('utf-8')
                if buffer:
                    reported_epochs = int(buffer)
                    print(f"Reported epochs: {reported_epochs}")
                    return reported_epochs
            except OSError as e:
                if e.errno == os.errno.EAGAIN or e.errno == os.errno.EWOULDBLOCK:
                    pass
                else:
                    raise
            finally:
                os.close(epochs_pipe_fd)
        else:
            pass
    


In [173]:
def run_optimization():
    suggested_value = optimizer.suggest()
    print(suggested_value)
    pass_values_to_interface(suggested_value)
    index=read_epochs_pipe()
    time.sleep(5)
    if index==-1:
        target_value=float('inf')
    else:
        target_value = cost_model(index)
    optimizer.register(suggested_value, pd.Series([target_value]))


n_iterations = 50
for i in range(n_iterations):
    print("epoch:",i)
    run_optimization()

epoch: 0
   arena_block_size avoid_flush_during_shutdown  \
0        1991677660                       false   

   blob_compaction_readahead_size  blob_file_starting_level  \
0                          536563                        11   

   blob_garbage_collection_age_cutoff  \
0                            0.291325   

   blob_garbage_collection_force_threshold check_flush_compaction_key_order  \
0                                 0.023024                             true   

   compaction_readahead_size  delayed_write_rate  \
0                  651882445           710324103   

   delete_obsolete_files_period_micros  ... max_write_buffer_number  \
0                        1034051847831  ...                      14   

  memtable_huge_page_size  memtable_prefix_bloom_size_ratio  \
0               225427145                          0.102194   

   memtable_whole_key_filtering  min_blob_size  paranoid_file_checks  \
0                          true            435                 false   


Reported epochs: 414
epoch: 7
   arena_block_size avoid_flush_during_shutdown  \
0        1627235802                        true   

   blob_compaction_readahead_size  blob_file_starting_level  \
0                          781747                        15   

   blob_garbage_collection_age_cutoff  \
0                            0.440633   

   blob_garbage_collection_force_threshold check_flush_compaction_key_order  \
0                                 0.391157                            false   

   compaction_readahead_size  delayed_write_rate  \
0                  419346315             6457557   

   delete_obsolete_files_period_micros  ... max_write_buffer_number  \
0                        2867904034630  ...                       7   

  memtable_huge_page_size  memtable_prefix_bloom_size_ratio  \
0                99690695                          0.228479   

   memtable_whole_key_filtering  min_blob_size  paranoid_file_checks  \
0                         false            734     

Reported epochs: 618
epoch: 14
   arena_block_size avoid_flush_during_shutdown  \
0         869675742                       false   

   blob_compaction_readahead_size  blob_file_starting_level  \
0                          583528                         1   

   blob_garbage_collection_age_cutoff  \
0                            0.406616   

   blob_garbage_collection_force_threshold check_flush_compaction_key_order  \
0                                  0.89739                             true   

   compaction_readahead_size  delayed_write_rate  \
0                   42927178           472851720   

   delete_obsolete_files_period_micros  ... max_write_buffer_number  \
0                        1495943300799  ...                      15   

  memtable_huge_page_size  memtable_prefix_bloom_size_ratio  \
0               506207746                          0.511506   

   memtable_whole_key_filtering  min_blob_size  paranoid_file_checks  \
0                          true            645    

KeyboardInterrupt: 

In [21]:
optimizer.get_observations()

,avoid_flush_during_shutdown,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,max_background_jobs,max_bytes_for_level_base,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,write_buffer_size,score
0,False,159,31,4,949973953,12.782758,6,11,333967540,13,377747531,2305.500000
0,False,21,17,19,673577312,19.535268,11,18,423980065,16,2343073,1919.000000
0,True,111,46,9,54418164,6.854681,1,4,169046098,7,745432179,1532.250000
0,True,25,32,6,209011921,2.913697,15,19,487697222,11,195788877,1308.000000
0,False,143,5,16,631752554,14.416207,5,1,236907193,3,669394079,1142.000000
0,False,80,42,1,891444195,16.673102,20,8,64968671,9,826678389,991.000000
0,True,188,22,11,461199343,9.648560,9,14,266674635,19,487307431,832.428571
0,True,2,39,18,1032807303,8.461841,14,19,499273414,6,459534972,942.666667
0,False,116,25,8,348636170,18.281314,4,2,225843376,17,925783720,681.571429
0,False,54,35,13,92192592,11.542537,18,6,4752980,15,562982466,801.142857


In [22]:
optimizer.get_best_observation()

,avoid_flush_during_shutdown,level0_file_num_compaction_trigger,level0_slowdown_writes_trigger,max_background_jobs,max_bytes_for_level_base,max_bytes_for_level_multiplier,max_subcompactions,max_write_buffer_number,target_file_size_base,target_file_size_multiplier,write_buffer_size,score
0,True,141,28,5,762133881,13.496323,19,3,517046316,17,571094216,302.375


In [23]:
optimum_value=optimizer.suggest()
pass_values_to_interface(optimum_value)

In [24]:
print(optimum_value)

   avoid_flush_during_shutdown  level0_file_num_compaction_trigger  \
0                         True                                 142   

   level0_slowdown_writes_trigger  max_background_jobs  \
0                              33                    5   

   max_bytes_for_level_base  max_bytes_for_level_multiplier  \
0                 764545077                       13.496323   

   max_subcompactions  max_write_buffer_number  target_file_size_base  \
0                  20                        3              517046316   

   target_file_size_multiplier  write_buffer_size  
0                           20          567098226  


In [ ]:
import pandas as pd
import plotly.express as px

data_url = '../../build/performance/performance_metrics.csv'
df = pd.read_csv(data_url, header=None, names=["Empty Reads", "Non-Empty Reads", "Range Reads", "Writes"]).iloc[-100:],

# Plot Empty Reads\n",
fig_empty_reads = px.line(df, x=df.index, y=\"Empty Reads\", title=\"Empty Reads\")\n",
fig_non_empty_reads = px.line(df, x=df.index, y=\"Non-Empty Reads\", title=\"Non-Empty Reads\")\n",
fig_range_reads = px.line(df, x=df.index, y=\"Range Reads\", title=\"Range Reads\")\n",
fig_writes = px.line(df, x=df.index, y=\"Writes\", title=\"Writes\")\n",

fig_empty_reads.update_xaxes(title_text='Iterations')
fig_empty_reads.update_yaxes(title_text='Time (ms)')
fig_non_empty_reads.update_xaxes(title_text='Iterations')
fig_non_empty_reads.update_yaxes(title_text='Time (ms)')
fig_range_reads.update_xaxes(title_text='Iterations')
fig_range_reads.update_yaxes(title_text='Time (ms)')
fig_writes.update_xaxes(title_text='Iterations')
fig_writes.update_yaxes(title_text='Time (ms)')

fig_empty_reads.show()
fig_non_empty_reads.show()
fig_range_reads.show()
fig_writes.show()

fig_empty_reads.write_image("../../build/performance/empty_reads_graph.png")
fig_non_empty_reads.write_image("../../build/performance/non_empty_reads_graph.png")
fig_range_reads.write_image("../../build/performance/range_reads_graph.png")
fig_writes.write_image("../../build/performance/writes_graph.png")

In [ ]:
hyper_parameters_integer = {
    "max_open_files":[-1,10000, -1],
    "max_total_wal_size":[0, 100000, 0],
    "delete_obsolete_files_period_micros":[0, 6 * 60 * 60 * 1000000, 6 * 60 * 60 * 1000000],
    "max_background_jobs":[2, 10, 2],
    "max_subcompactions":[1, 100, 1],
    "compaction_readahead_size":[0, 1000000, 0],
    "writable_file_max_buffer_size":[0, 1024 * 1024, 1024 * 1024],
    "delayed_write_rate":[0, 1000000, 0],
    
    "max_write_buffer_number":[1, 10, 2],
    "inplace_update_num_locks":[0, 10000, 0],
    "memtable_huge_page_size":[0, 1000000, 0],
    "arena_block_size":[0, 1000000, 0],
    "level0_slowdown_writes_trigger":[0, 100, 20],
    "level0_stop_writes_trigger":[0, 100, 36],
    "target_file_size_base":[0, 100000000, 64 * 1048576],
    "target_file_size_multiplier":[0, 100, 1],
    "max_compaction_bytes":[0, 1000000000, 64 * 1048576 * 25],
    "soft_pending_compaction_bytes_limit":[0, 1000000000, 64 * 1073741824],
    "hard_pending_compaction_bytes_limit":[0, 1000000000, 256 * 1073741824]                          
}
                                           
hyper_parameters_float ={
    "memtable_prefix_bloom_size_ratio":[0.0, 1.0, 0.0],
    "max_bytes_for_level_multiplier":[0.0, 100.0, 10.0]
}
                                           
hyper_parameters_catagorical ={
    "avoid_flush_during_shutdown":[[True, False],False],  
    "memtable_whole_key_filtering":[[True, False],False],
   "compression_per_level":[["kNoCompression", "kSnappyCompression", "kZlibCompression", "kBZip2Compression", "kLZ4Compression", "kLZ4HCCompression", "kXpressCompression", "kZSTD"], "kNoCompression"]

}